# Install KB-LAB client

```bash
pipenv install kblab-client
```

In [ ]:
import os
import dotenv
from kblab import Archive
from json import load

dotenv.load_dotenv(dotenv_path='~/.vault/.kblab.env')

a = Archive('https://betalab.kb.se/', auth=(os.environ['KBLAB_USER'], os.environ['KBLAB_PASSWD']))

query = { "tags": "issue", "meta.created": "1927" }
query = { "tags": "protokoll" }

a.search(query)


In [ ]:
result = a.search({ "tags": "protokoll" }, max=2)

for package_id in result:

    package = a.get(package_id)

    print(dir(package))
    print(package.list())
    
    for file in package:
        content = package.get_raw(file).read()
        
        print(dir(content))
        print(dir(file))

        break
    break



In [ ]:

for package_id in a.search({ 'label': 'AFTONBLADET' }, max=2):
    print(package_id)
    p = a.get(package_id)

    #print(p.label)



In [ ]:
import os
import zipfile
import fnmatch
import re

from kblab import Archive
#from json import load

# from utility import alto2text
import xmltodict

def kblab_connect():

    kb_archive = Archive('https://betalab.kb.se/', auth=(os.environ['KBLAB_USER'], os.environ['KBLAB_PASSWD']))

    return kb_archive

def kblab_download_package(kb_archive, package_id, target_archive, excludes=None):

    excludes = excludes or []

    package = kb_archive.get(package_id)

    if 'content.json' not in package:
        print("warning: {} has no content".format(package_id))
        return

    text_items = []
    text_item_id = -1
    for filename in sorted(package.list()):

        if any(fnmatch.fnmatch(filename, pattern) for pattern in excludes):
            # if os.path.splitext(filename)[1] in excludes:
            print("skipping: {}".format(filename))
            continue

        content = package.get_raw(filename).read()

        target_archive.writestr(os.path.join(package_id, filename), content, zipfile.ZIP_DEFLATED)

        print("stored: {}".format(filename))

        m = re.match(r'prot_(\d{4})__(\d+)\-(\d+)\.xml', filename)
        if m is not None:

            year, id, page = m.groups()

            assert package_id == "prot-{}--{}".format(year, id)

            if (text_item_id + 1) != int(page):
                print("warning: page(s) with no XML found {} expected {}".format(int(page), previous_page + 1))

            page_content = xmltodict.parse(content)
            assert isinstance(page_content, dict) and 'alto' in page_content
            package_documents.append(page_content)

    #if len(pages) > 0:
    #    target_archive.writestr(os.path.join(package_id, "{}.txt".format(package_id)), '\n'.join(pages), zipfile.ZIP_DEFLATED)

    return package_documents

kb_archive = kblab_connect()
target_folder = "/home/roger/tmp"
target_filename = os.path.join(target_folder, "riksdagens_protokoll.zip")
query = { "tags": "protokoll" }
max_count = 2

with zipfile.ZipFile(target_filename, "w") as target_archive:

    for package_id in kb_archive.search(query, max=max_count):

        package = kb_archive.get(package_id)

        package_documents = kblab_download_package(kb_archive, package_id, target_archive, ["*.jpg", "*.jb2e"])



In [ ]:
document = pages[0]
layout = page['alto']['Layout']
for page in layout:
    pass

In [ ]:
dir(package)

In [ ]:
dir(package.description)